# Lec5 統計の基礎

## 講義の内容
- 統計とは
  - 母集団と標本
  - 記述統計量
  - 正規分布、様々な確率分布
- 推定
  - 点推定、区間推定と信頼区間
- 仮説検定
  - 帰無仮説、対立仮設、有意水準、相関


統計分布について一通り確率密度関数から検定までが実装されている．scipy.statsを使います。

参考：
https://github.com/rasbt/data-science-tutorial

## 要約統計量 (Summary Statistics)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

Irisデータセットをロードし、'sepal_length'(がく片の長さ cm)の分布をヒストグラムで確かめます。

https://raw.githubusercontent.com/miwamasa/DataScience2024/main/notebooks/datasets/iris/iris.csv

In [ ]:
# read dataset
df = pd.read_csv('https://raw.githubusercontent.com/miwamasa/DataScience2024/main/notebooks/datasets/iris/iris.csv')

def histo():
    # create histogram
    bin_edges = np.arange(0, df['sepal_length'].max() + 1, 0.5)
    fig = plt.hist(df['sepal_length'], bins=bin_edges)

    # add plot labels
    plt.xlabel('sepal length')
    plt.ylabel('count')



histo()

'sepal_length'の方がfloat64であることを確かめます。(N.A等が入ってないかどうか)

In [ ]:
x = df['sepal_length'].values
x.dtype

平均を計算します

In [ ]:
sum(i for i in x) / len(x)

numpyのmean/0関数でも計算できますが、値は同じではありませんん。

In [ ]:
x_mean = np.mean(x)
x_mean

ヒストグラムに、平均の線（縦線）を重ね書きします。

In [ ]:
histo()
plt.axvline(x_mean, color='red')

分散を計算します。最初は、公式どおりに計算します


In [ ]:
sum([(i - x_mean)**2 for i in x]) / (len(x) - 1)

numpyのvar/2を使っても計算できます。ddofは

Bessel's correction(nの代わりにn-1で割ること)を考慮している

In [ ]:
var = np.var(x, ddof=1)
var

In [ ]:
df['sepal_length'].var() # note that Bessel's correction is the default

それでは平均±分散を縦線で重ねます

In [ ]:
histo()
plt.axvline(x_mean + var, color='darkorange')
plt.axvline(x_mean, color='red')
plt.axvline(x_mean - var, color='darkorange')

標準偏差：

　numpyのstd関数でも計算できます。Bessel's correction が入ってます

In [ ]:
(sum([(i - x_mean)**2 for i in x]) / (len(x) - 1))**0.5

In [ ]:
np.sqrt(np.var(x, ddof=1))

In [ ]:
std = np.std(x, ddof=1)
std

In [ ]:
df['sepal_length'].std() # note that Bessel's correction is the default

In [ ]:
histo()
plt.axvline(x_mean + std, color='darkorange')
plt.axvline(x_mean, color='red')
plt.axvline(x_mean - std, color='darkorange')

Min/Max

In [ ]:
np.min(x),np.max(x)

モード（最頻値):

いったんlistに変換すると list.count(値)で、その値をもつデータ数を数え上げられます

In [ ]:
lst = list(x)
lst.count(5.7)



関数 max(set(list_name), key = list_name.count) は、指定されたリストで最大回数発生する要素を返します。

> "max(list, key=func) 関数は、イテレーブルオブジェクト (リスト、タプル、辞書など) から最大値を見つけ、それを返します。オプション引数 key を指定することで、各要素に対して func 関数を適用し、その結果に基づいて最大値を判断することができます"(from Gemini)

In [ ]:
lst = list(x)
mode = max(set(lst), key=lst.count)
mode

最頻値に何個データがあったか、

In [ ]:
lst.count(mode)

stasのmode関数でも求められます

In [ ]:
stats.mode(x)

25パーセント点と75パーセント点:

- まずはxをソートしてyを得ます
- y.shape[0]は、yの要素数を返します。要素数の25%/75&のところの値を得ます

In [ ]:
y = np.sort(x)
percentile_25th = y[round(0.25 * len(y)) + 1]
percentile_25th

In [ ]:
percentile_75th = y[round(0.75 * len(y)) - 1]
percentile_75th

numpyのpercentile関数を使ってもば求まります。

In [ ]:
np.percentile(x, q=[25, 75], interpolation='lower')

ご参考
> "interpolation オプションに 'lower' を指定すると、<略>、下位側に向かって最も近い値 を選択するようになります。。"(by Gemini)

pandasのDataFrameのquntile関数を使っても求められます

In [ ]:
df['sepal_length'].quantile(0.25, interpolation='lower')

In [ ]:
df['sepal_length'].quantile(0.75, interpolation='lower')

25パーセント点と75パーセント点をプロットしてみます

In [ ]:
histo()
plt.axvline(percentile_75th, color='darkorange')
plt.axvline(np.mean(x), color='red')
plt.axvline(percentile_25th - var, color='darkorange')

中央値 (50パーセント点でもある):

In [ ]:
x = np.sort(x)

tmp = round(0.5 * x.shape[0])

if x.shape[0] % 2:
    median = x[tmp - 1]
else:
    median = x[tmp - 1] + (x[tmp] - x[tmp - 1]) / 2.

median

numpyにはmedian関数がある

In [ ]:
np.median(x)

In [ ]:
histo()
plt.axvline(median, color='yellow')
plt.axvline(np.mean(x), color='red')

## 仮説検定入門
hypothesis-testing_1.ipynb　より

### Case 1 -- 並べ替え(permutating)サンプリングによる、帰無仮説に基づいたシミュレーションの利用 (2-sample test)
- 2標本間の平均値差の検定（並べ替え検定)
- 帰無仮説: ２分布の間での平均は同一である(「2 つの母集団は同じもの
である」）
- 2分布の平均には差がある（経験的平均）
- 帰無仮説が正しいならば、経験的平均は確率的にそうなったに違ない、じゃあ、この平均の差はどれぐらいの確率で起こるものなのかを計算する。
- データを並べ替えて (permute the data) ２つに分けて違いがあるかどうかを検証する。(帰無仮説に基づいて新しいデータを生成しているともいえる)
- 参考：https://www.mizumot.com/files/permutation.pdf

Irisデータを使う。setosaとversicolorのsepal_widthのリストを得る(x1,x2)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/miwamasa/DataScience2024/main/notebooks/datasets/iris/iris.csv')
x1 = df.loc[df['species'] == 'setosa', 'sepal_width'].values
x2 = df.loc[df['species'] == 'versicolor', 'sepal_width'].values

In [ ]:
for d in (x1, x2):
    print('Mean:', np.mean(d))
    print('Std.Dev.:', np.std(d, ddof=1))

the difference in means (our test statistic) of permuted samples should be zero if the null hypothesis is true

２つのサンプル集合をくっつけて、並べ替えて、２つに分割して返します。

In [ ]:
def generate_permutation_samples(sample1, sample2, seed=None):
    rng = np.random.RandomState(seed)
    sample = np.concatenate((sample1, sample2))
    sample_perm = rng.permutation(sample)
    sample1_perm = sample_perm[:sample1.shape[0]]
    sample2_perm = sample_perm[sample1.shape[0]:]
    return sample1_perm, sample2_perm

x1,x2から並び替えを行いx1_perm,x2_permを得てこの差を perm_mean_diffsの要素として格納することを1000回行う。

In [ ]:
num_permutations = 1000

perm_mean_diffs = np.zeros(num_permutations)
for i in range(num_permutations):
    x1_perm, x2_perm = generate_permutation_samples(x1, x2, seed=i)
    perm_mean_diffs[i] = np.mean(x1_perm) - np.mean(x2_perm)

もともとx1,x2の分布の違いは？

In [ ]:
plt.hist(x1, label='x1', alpha=0.5, color='blue')  # x1のヒストグラム
plt.hist(x2, label='x2', alpha=0.5, color='red')   # x2のヒストグラム
plt.title('x1 vs x2')
plt.xlabel('width')
plt.ylabel('frequency')
plt.legend()

経験的平均の差をもとめる。差がある

In [ ]:
empirical_mean_diff = np.mean(x1) - np.mean(x2)
empirical_mean_diff

経験的な平均の差は、確率的にそれが得られたものとすると、じゃあ、そうじゃないデータを前提（帰無仮説）とすると経験的平均の差はどれだけ、ありえないのかを検定する。

perm_mean_diffsの分布（リサンプリングした２つの分布の平均の差の分布）と、経験的平均（オレンジ）の重ね書きで可視化する

In [ ]:
plt.hist(perm_mean_diffs, bins=20)
plt.axvline(empirical_mean_diff, color='orange')

p値は、、0.0である

In [ ]:
p_value = np.sum(perm_mean_diffs >= empirical_mean_diff) / len(perm_mean_diffs)
print('P-value', p_value)

- 経験的平均(オレンジの縦線)から左（あるいは右）の領域がp値である
- オレンジの縦線から右の領域の面積が0.0であるから、帰無仮説が正しいならば、標本の平均の差を観測する可能性もっと極端なことは0%らしい、ことがわかる。
- すなわち、帰無仮説はrejectされる。帰無仮説が正しければ、ここで観測が期待されるより、統計的に優位に異なるデータが得られるはずである

### Case 2 -- ブートストラップ法(bootstraping)による、帰無仮説に基づいたシミュレーションの利用 (1-sample test)


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/miwamasa/DataScience2024/main/notebooks/datasets/iris/iris.csv')
x1 = df.loc[df['species'] == 'setosa', 'sepal_width'].values
x1_mean = np.mean(x1)
x1_mean

- 帰無仮説：x1とx2は同じ平均値を持つ（分布は同じとは限らない）
- x2からは標本データは得られず、平均だけが得られるとする
- 有意水準を$\alpha=0.05$とする

In [ ]:
x2_mean = 3.55

1. x1を正規化し、x2と同じ平均を持つようにする
2. ブートストラップされたレプリカを得る
3. p値を計算

In [ ]:
# 1) normalize x1 so that it has the same mean as x2
normed_x1 = x1 - np.mean(x1) + x2_mean

# 2) draw bootstrap replicates
def bootstrap_means(x, n_bootstrap_samples, seed=None):

    rng = np.random.RandomState(seed)
    sample_means = np.zeros(shape=n_bootstrap_samples)

    for i in range(n_bootstrap_samples):
        boot_sample = rng.choice(x, size=x.shape[0], replace=True)
        # replicate is a general term for a statistic computed
        # from a bootstrap sample
        bootstrap_replicate = np.mean(boot_sample)
        sample_means[i] = bootstrap_replicate
    return sample_means

num_replicates = 1000
bs_replicates = bootstrap_means(x1, n_bootstrap_samples=num_replicates, seed=123)

# 3) compute p-value
p_value = np.sum(bs_replicates >= x2_mean) / num_replicates

print('p-value', p_value)

なんとなく、x1,x2の平均はそんなに違わない気もしますが、ブートストラップで増やしてみると。

In [ ]:
plt.hist(x1, label='x1', alpha=0.5, color='blue')  # x1のヒストグラム
#plt.hist(bs_replicates, label='x2', alpha=0.5, color='red')   # x2のヒストグラム
plt.axvline(x2_mean, color='orange')
plt.axvline(x1.mean(), color='red')

at $\alpha=0.005$, we can reject the null hypothesis that the two variables are from the same distribtion

In [ ]:
plt.hist(bs_replicates, bins=20)
plt.axvline(x2_mean, color='orange')

## 分布の件

# 正規分布を知る

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.stats
from scipy.stats import norm  # normal distribution, 正規分布

import matplotlib.pyplot as plt
%matplotlib inline

FLAG_fig = True

正規分布のグラフ pdf（probability density function）

In [ ]:
m = 5   # 平均値
std = 2 # 標準偏差
std2 = 4
x = np.arange( -5, 15, 0.01)
y = norm.pdf(x, loc=m, scale=std)
y2 = norm.pdf(x, loc=m, scale=std2)
#y = (1 / np.sqrt(2 * np.pi * std*std ) ) * np.exp(-(x-m) ** 2 / (2 * std*std) ) #正規分布の式

fig = plt.subplots(figsize=(8,3))
plt.plot(x,y,label='std=2')
plt.plot(x,y2,label='std=4')
plt.xlabel('x')
plt.ylabel('probabiity density')
plt.legend()
if FLAG_fig: plt.savefig('fig_PROB_NormalDist.png')

#### パーセント点と確率の計算
パーセント点（pp:percent point)$z_{\alpha}$ を求めるには，$1-\alpha$をppf ( percent point function )に与える。<br>
確率を求めるには，パーセント点$z_{\alpha}$をcdf (cumulative density function)　に与える。<br>
なお，標準正規分布（平均値 $m = 0$,  標準偏差 $\sigma=1$ ）を対象とする<br>

#### 片側の場合
標準正規分布$N(0,1)$のグラフで，$z_{\alpha} \le x < \infty$ の確率 $\alpha$（面積）をユーザが与える。このとき，  
$- \infty < x \le z_{\alpha} $の確率（面積）は $1-\alpha$となる。
この値をscipy.stats.norm.ppf に与えれば，$z_{\alpha}$が求まる。

In [ ]:
m = 0
std = 1
alpha = 0.05
prob = 1 - alpha
z_alp = norm.ppf(prob, loc=m, scale=std)
print('パーセント点＝', z_alp) #

In [ ]:
# グラフ書いてみるよ
m = 0
std = 1
alpha = 0.05

x = np.arange( -3, 3, 0.01)
y = norm.pdf(x, loc=m, scale=std)

fig = plt.subplots(figsize=(5,3))
plt.plot(x,y,label='std=1')

prob = 1 - alpha
z_alp = norm.ppf(prob, loc=m, scale=std)
plt.axvline(z_alp, color='orange',label="z_alp={0}".format(z_alp))
plt.title("Percent point z_alp for alpha=0.05")
plt.legend()
if FLAG_fig: plt.savefig('fig_PROB_PercentPoint.png')

isf (Inverse survival function，生存関数の逆関数）を用いると，1-alphaの計算が不要で，上記と同じパーセント点を得る

In [ ]:
z_alp2 = norm.isf(alpha, loc=m, scale=std)
print('isfを用いたパーセント点', z_alp2)

In [ ]:
m = 0
std = 1.0
z_alp = 1.96
prob = norm.cdf(z_alp, loc=m, scale=std)
print('確率＝',prob)

#### 両側の場合
区間[a b]の確率を求める。
1. $- \infty < x \le z_{b} $の確率$P_a$を求める  
2. $- \infty < x \le z_{a} $の確率$P_b$を求める
3. $P_a - P_b$を計算

In [ ]:
za = 1.96
zb = -1.96
pa = norm.cdf(za, loc=0, scale=1) # loc is mean
pb = norm.cdf(zb, loc=0, scale=1) # scale is standard deviation
p = pa - pb
print('p=',p)

In [ ]:
# グラフ書いてみるよ
za = 1.96
zb = -1.96
x = np.arange( -3, 3, 0.01)
y = norm.pdf(x, loc=0, scale=1)

fig = plt.subplots(figsize=(5,3))
plt.plot(x,y,label='std=1')

pa = norm.cdf(za, loc=0, scale=1) # loc is mean
pb = norm.cdf(zb, loc=0, scale=1) # scale is standard deviation
p = pa - pb

plt.axvline(za, color='orange',label="za={0}".format(za))
plt.axvline(zb, color='yellow',label="zb={0}".format(zb))
plt.title("Probability for p=pa-pb={0}".format(p))
plt.legend()
if FLAG_fig: plt.savefig('fig_PROB_ProbFromPercentPoint.png')

#### 検定では，$\alpha$が初めに与えられる。
これに基づき，片側（z_alp），両側（za, zb）を求めることが多い。  
片側の場合は既に述べた。  
両側の場合，正規分布が対称で，かつ，（za, zb）が原点を中心とした左右対称という前提があり，この場合，intervalを用いることができる。<br>
下記の例では，両側にz_alp/2=0.025 があり，この二つを足して0.05となることに留意。

In [ ]:
za,zb = norm.interval(confidence=0.95, loc=0, scale=1)
print('za=',za,'  zb=',zb)

横軸が $x$,  縦軸が頻度（データ数Nが多いほど，縦軸のスケールも大きくなる）<br>
・平均値を中心とした分布となる<br>
・データ数100程度では，正規分布の形とは言えず，データ数を非常に多くして，ようやく理想形に近くなる<br>
・標本平均値，標本標準偏差もデータ数がかなり多くないと，真値に近づかない<br>
・標準偏差の計算 x.std(ddof=1) のddof=1は“不偏標準偏差”を求めるとき，すなわち，1/(N-1)という除算を行う<br>
・このddofを指定しないと， 1/N を用いた計算を行い，不偏とならない<br>
・標準偏差はばらつきの指標となる

In [ ]:
np.random.seed(123) # scipyと共通，乱数発生の再現性を得る

mean = 2.0 # mean, 平均値
std = 3.0  # standar deviation, 標準偏差
#for N in [100, 10000]:
for N in [100]:
    x = scipy.stats.norm.rvs(loc=mean, scale=std, size=N) # rvs:Random variates
    print('N = %d  mean = %f  std = %e' % (N, x.mean(), x.std(ddof=1)))
    plt.figure()
    plt.hist(x, bins=20)
    plt.title('$N = %i$' % (N) )

plt.show()

#### 中心極限定理
一様乱数を用いる，この区間[a,b]の平均値は$\mu = \frac{a+b}{2}$, 分散$\sigma^2 = \frac{(b-a)^2}{12}$である。
このn個（幾つかの値）の標本平均をN個（これは一定とする）発生させ，その分布を見ると，正規分布$N(\mu, \sigma^2/n )$に近づく。ここでは，標準正規分布に正規化する。
他の分布でも試してみられたい。

In [ ]:
N = 2000
y= np.zeros(N)
for n in [5, 5000]:
    for i in range(N):
        x = scipy.stats.uniform.rvs(size=n)
        y[i] = (x.mean() - 1/2)/(np.sqrt(1/12)/np.sqrt(n))
    plt.hist(y, bins=20, range=(-4,4), density=True)

xx = np.arange(-4, 4, 0.01)
nrm = scipy.stats.norm.pdf(xx, loc=0.0, scale=1.0 )
plt.plot(xx, nrm, c='k')


### ポアソン分布
$$
P(X=k) = \exp (-\lambda t) \frac{(\lambda t)^k}{k !}
$$
scipy.stats.poisson  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy.stats import poisson
from scipy.stats import uniform

import matplotlib.pyplot as plt
%matplotlib inline


### 確率質量関数（pmf: Probability mass function）のグラフ
離散確率変数の場合の呼称 ，連続確率変数の場合は確率密度関数 (pdf: Probability density function)

In [ ]:
fig = plt.subplots(figsize=(8,4))
k = np.arange(0,16)

for lamb in range(1,6):
    p = poisson.pmf(k, lamb)
    plt.plot(k, p, label='lamb='+str(lamb))

plt.xlabel('k')
plt.ylabel('Probability mass function')
plt.legend()


#### 例：交通事故問題
交通事故　平均2.4件／日のとき，交通事故が2件／日　以下となる確率を求める。

In [ ]:
lamb = 2.4
psum = 0
for k in [0, 1, 2]:
    p = poisson.pmf(k, mu=lamb)
    psum = psum + p
print('sum of p =',psum)

#### ポアソン到着モデル
$$
  t_{arrive} = -\frac{1}{\lambda} \log_{e} P_{arrive} (T)
$$

In [ ]:
np.random.seed(123)

Num=30 # 　Num人分の到着時刻を得る
t_arrive = np.zeros(Num)
lamb = 1

sum = 0.0
for i in range(Num):
    sum = sum - (1/lamb) * np.log( uniform.rvs(size=1) )
    t_arrive[i] = sum

fig, ax = plt.subplots(figsize=(6,3))
ax.vlines(t_arrive, ymin=0, ymax=1)
ax.set_xlabel('time[k]')
plt.tight_layout()


In [ ]:
t_arrive

In [ ]:
Num=30 # the number of arraivl, Num人分の到着時刻を得る
t_arrive = np.zeros(Num)
lamb = 1

sum = 0.0
for i in range(Num):
    sum = sum - (1/lamb) * np.log( uniform.rvs(size=1) )
    t_arrive[i] = sum

fig, ax = plt.subplots(figsize=(6,3))
ax.vlines(t_arrive, ymin=0, ymax=1)
ax.set_xlabel('time[k]')
plt.tight_layout()


## 平均値の検定　（母分散が未知） t検定
$$
t = \frac{\hat{\mu} - \mu_0}{\sqrt{\sigma^2 \big/ N}}
$$
上記の$t$検定量は自由度$ df = N - 1 $の$t$分布に従う（ $df$: Degree of Freedom）<br>
scipy.stats.t https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html <br>

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline


## 累積確率(CDF:cumulative distribution function)

In [ ]:
x = np.linspace(-4, 4, 100)
plt.plot(x, stats.norm.pdf(x))
#plt.plot(x, stats.t.pdf(x,99))
plt.xlabel('x')
plt.ylabel('P')
plt.title('Normal Distribution')

In [ ]:

#y_cdf = stats.t.cdf(x,100-1)
y_cdf = stats.norm.cdf(x)
plt.plot(x, y_cdf)
plt.xlabel('x')
plt.ylabel('P')
plt.title('Cumulative Distribution Function')

In [ ]:
len(x)

In [ ]:
stats.t.cdf(0.53,100-1)-stats.t.cdf(0,100-1)

In [ ]:
1-stats.norm.cdf(1.53)

In [ ]:
stats.norm.cdf(0.53)-stats.norm.cdf(0)

#### 例１：あるクラス，テストの平均点と補講の効用（片側検定）

In [ ]:
data = np.array([1, -1, -2, 3, -1, 5, 4, 0, 7, -1])
m = np.average(data) # mean
s = np.std(data, ddof=1) # std, ddof=1 : unbiased
N = len(data) # the number of sample
print(m,s,N)

In [ ]:
alp = 0.05
# 自由度N-1のt分布で、alphaに対するパーセント点を求める(1-alpa)をppf(percent point function)に入れて求める->talp
talp = stats.t.ppf((1-alp),N-1)
print('talp (alpha=0.05, df=%d) =%f' %((N-1),talp))

m0 = 0 # null hypothesis
# 母関数の平均が0が帰無仮説
# t値は、は talpより大？小？
t = (m-m0)/(s/np.sqrt(N))
print('t=', t)

talp > t より，H0は棄却できない。
通常は，talp, t のようなpp(percent point)を求めるよりは，
p値(p value)を求める。これを次に示す

In [ ]:
prob = stats.t.cdf(t,N-1)
print('p value=',1-prob)

有意水準を0.05 とおくと，これよりp valueの方が大きいので，H0を棄却できない。

#### 例2：精密部品の直径（両側検定）

In [ ]:
data2 = np.array([1.5399, 1.5390, 1.5399, 1.5395, 1.5400, 1.5390, 1.5399, 1.5399])
m = np.mean(data2)        # mean
s = np.std(data2, ddof=1) # std, ddof=1 : unbiased
N = len(data2)            #
df = N - 1               # DoF (degree of freedom)
m0 = 1.54                 # H0 (null hypothesis)
print('sample mean =',m,'  std = ',s,' The number of data = ',N)

In [ ]:
t = (m-m0)/(s/np.sqrt(N))   # サンプルから求まるpp値
prob = stats.t.cdf(t, df)
if t >=  0:
    p = 1 - prob
else:
    p = prob

print('t = ',t)
print('p value =',2*p)

In [ ]:
data2 = np.array([1.5399, 1.5390, 1.5399, 1.5395, 1.5400, 1.5390, 1.5399, 1.5399])
m = np.mean(data2)        # mean
s = np.std(data2, ddof=1) # std, ddof=1 : unbiased
m0=1.54
N = len(data2)
t = (m-m0)/(s/np.sqrt(N))   # サンプルから求まるpp値
prob = stats.t.cdf(t, N-1)
p= (1-prob) if t >= 0 else prob
pvalue=2*p
pvalue

In [ ]:
prob,t,m,s

注意：上記のように，tが負の値をとるときは，p = prob とする。  
$\alpha=0.05$と置くならば，p value $< \alpha$ よりH0を棄却

実は，次の関数を用いると計算は楽<br>
t, p = scipy.stats.ttest_1samp(data, m0)<br>
data: 1群のサンプルデータ<br>
m0: 帰無仮説H0で仮定した平均値<br>
t: t値<br>
p: p値，両側検定を前提としている。片側検定ではこの半分の値を用いる<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html

In [ ]:
t, p = stats.ttest_1samp(data2, m0)
print('t = ',t)
print('both side p = ',p)
print('one  side p = ',p/2)

2024.5.6追記

【例題4.5】

前出アルファ社の精密部品について、規格では分散は$1\times 10^{-7}$mm以下としている。このとき母標準偏差は、この規格からずれているか有意水準5%で検定する。$H_0$を$\sigma^2 =1 \times 10^{-7}$,$H_1$を$\sigma^2 \gt 1 \times 10^{-7}$とする


In [ ]:
uvar = np.var(data2,ddof=1)
var, N = 1.e-7, len(data2)
X = (N-1)*(uvar/var)
pval=stats.chi2.sf(X,df=N-1)
print('p value=', pval)